# Lesson 2: Scientific Data Formats and Advanced Plotting

Author: Rebekah Esmaili (rebekah.esmaili@gmail.com)
 
---

## Lesson Objectives
* You will learn to:
    * Import relevant packages for scientific programming
    * Read netCDF and GRIB2 data
    * Creating plots and maps
   

![](img/flowchart.png)




## Importing NetCDF files

NetCDF and HDF are self-describing formats, which are structured binary data files and useful for storing other big datasets. Computationally, it is faster to read in binary-based datasets than text, which needs to be parsed before being stored into a computer’s memory. Because the files are more compact, they are cheaper to store large, long-term satellite data. Furthermore, information about the data ("metadata") can be stored inside the file themselves.

Datasets:
* JRR-AOD_v2r3_j01_s202009152044026_e202009152045271_c202009152113150_thinned.nc: A netCDF file that contains Aerosol Optical Depth (AOD) retrieved from a Suomi NPP overpass on 2020 9 Aug.  For this workshop, unused fields were removed.
* gfs_3_20200915_0000_000.grb2: A GRIB2 file that contains GFS analysis
* MOP03JM-201811-L3V95.6.3_thinned.nc: The Nov 2018 CO monthly mean from the Measurement of Pollution in the Troposphere (MOPITT), which is an instrument on the Terra satellite.
    * NOTE: For this tutorial, the file was converted to a netCDF4 file and unused variable fields were removed. The original file is HDF5 MOP03JM-201811-L3V95.6.3.he5 and can be obtained from https://earthdata.nasa.gov/.
*  [NOAA Extended Reconstructed SST version 5 dataset (ERSST)](https://psl.noaa.gov/data/gridded/data.noaa.ersst.v5.html). Shows the global monthly mean ocean surface temperature from 1854-present using data collected from ocean buoys, ships, and climate modeled data.

Many environmental dataset names are quite long. However, the dataset name is encoded to give us information about the contents. For example:

```
JRR-AOD_v2r3_j01_s202009152044026_e202009152045271_c202009152113150.nc
```
You can learn several important features of the dataset without opening it:

* Prefix indicates the mission (JRR, for JPSS Risk Reduction)
* Product (Aerosol Optical Depth, or AOD), algorithm version
* Revision number (v1r1)
* Satellite source (j01 for JPSS-1/NOAA-20)
* Start (s), end (e), and creation (c) time, which are each followed by the year, month, day, hour, minute, and seconds (to one decimal place). 

First, import three commonly used packages in Python:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

To begin, you need to first import [xarray](http://xarray.pydata.org/en/stable/io.html) which is tailored to open netCDF4 files and work with large arrays (like numpy and pandas). The [netCDF4 package](https://unidata.github.io/netcdf4-python/netCDF4/index.html) can also be used to import files.

In [ ]:
import xarray as xr

Use the Dataset function to import the above dataset.

In [ ]:
fname='data/JRR-AOD_v2r3_j01_s202009152044026_e202009152045271_c202009152113150_thinned.nc'
aod_file_id = xr.open_dataset(fname)

If you print the contents of the file_id variable, you will get a long list of the global attributes, variables, dimensions, and much more.

In [ ]:
aod_file_id

The output above is worth inspecting. Inside Jupyter Notebooks, xarray allows you to inspect the file contents. Clicking on the arrows will show a preview of the metadata. Note that you can also use tools like [Panoply](https://www.giss.nasa.gov/tools/panoply/) to inspect the contents of the netCDF file outside of Python.

* __Dimensions__: The dimensions are named Rows and Columns, which are respectively 768 and 3200.

* __Coordinates__: The coordinates are Latitude and Longitude. These are both two dimensions.

* __Variables__: This file has only one variable, which is AOD550. It's dimensions are also Rows and Columns.

* __Attributes__: netCDF4 [CF-1.5 conventions](https://cfconventions.org/). Some of the information that we saw in the file name is also present: this product is the *JPSS Risk Reduction Unique Aerosol Optical Depth* (title) *Level 2* product (processing_level) and the data was collected from the *NOAA-20* (satellite_name) *VIIRS* instrument (instrument_name). The *start* (time_coverage_start) and *end* times (time_coverage_end) metadata fields are consistent with the filename. I recommend that you read netCDF file header contents, especially the first time you are working with new data. 

AOD is a unitless measure of the extinction of solar radiation by particles suspended in the atmosphere. High values of AOD can indicate the presence of dust, smoke, or another air pollutant while low values indicate a cleaner atmosphere.

Xarray syntax will resemble both Pandas and Numpy. Unlike numpy, N-D arrays can be labeled. Instead of having to remember indices numbers, we can extract elements using their coordinate or variables names.

Below I'll extract three important variables: AOD550, Latitude, and Longitude:

In [ ]:
AOD_550 = aod_file_id['AOD550']
AOD_lat = aod_file_id['Latitude']
AOD_lon = aod_file_id['Longitude']

Let's print AOD_550 below. This variable contains only a portion of the original data array:

In [ ]:
AOD_550

Xarray uses NumPy as a dependency so so we can use numpy functions like *.mean()*. First we have to make sure it's in the right format. If you check the type of *AOD_550*, you can see it's a *numpy.ndarray.*

In [ ]:
type(AOD_550.values)

Xarray handles missing data automatically, so if we do statistics on the array, it will not include them:

In [ ]:
avgAOD = AOD_550.mean()
print(avgAOD)

---
**Exercise 1**: Importing netCDF files
1. Open the file "MOP03JM-201811-L3V95.6.3_thinned.nc" using the xarray library
2. Print the variable names
3. What are the dimensions?
---

**Solution:**

## Importing GRIB2 files

GRIB2 files is a binary datasets that take on a table-driven code form. "Table driven" means that the files require external tables to decode the data type. Thus, they are not self-describing. These files follow a methodology of encoding binary data and not a distinct file type. Binary Universal Form for the Representation of meteorological data (BUFR) and GRIdded Binary (GRIB) are two common table-driven formats in Earth Sciences. 

American NWS models (e.g. GFS, NAM, and HRRR) and the European (e.g. ECMWF) models are stored in GRIB2. While they share the same format, there are some differences in how each organization stores its data. GRIB2 are stored as binary variables with a header describing the data stored followed by the variable values.

Currently, some of the GRIB2 decoders have problems parsing the American datasets because the American models have multiple pressure dimensions (depending on the variable) while the European models have one. Still, there are ways the data can be inspected by using the pygrib and cfgrib packages.

In [ ]:
import pygrib

The pygrib package (Unidata) has an interface between Python and the GRIB-API (ECMWF). ECMWF has since ended support for the GRIB-API as the primary GRIB2 encoded and decoder and now use ecCodes. However, the package is still maintained by the developer (https://jswhit.github.io/pygrib/) and is useful for parsing NCEP weather forecast data.

In [ ]:
filename = 'data/gfs_3_20200915_0000_000.grb2'
gfs_grb2 = pygrib.open(filename)

This opens the file, but does not extract the elements:

In [ ]:
type(gfs_grb2)

Below is a *for loop* in Python. The code block below will iterate over each item in the open dataset and append (using *.append*) them to a list (*records*). Note that if you run this command again, you will read to the end of the file, so there will be no result. You will have to re-open the command and re-run the block below.

You can check the size of the final list using *len(messages)*:

In [ ]:
records = []
for grb in gfs_grb2:
    records.append(str(grb))
    
len(records)

There are 522 individual data product definition in this file, so first let’s inspect the contents of one line to start:

In [ ]:
records[12]

From the output above, you can see that the colons (:) separate the sections of the product definition in this GRIB2 message. The elements are *index* (1), *variable name* and *units* (2-3), and *spatial*, *vertical*, and *temporal* definitions (4-8). There is one record for each *pressure level* and *time*. You can then extract all variables using the *.select(name=\[variable\])* command. Below, you select all the Temperature records (there are 46, which you can see by using the *len(temps)* command). Since it is a long list, you are only printing some of these below:

In [ ]:
temps = gfs_grb2.select(name='Temperature')

If you want to extract temperature at 85000 Pa, you can use the index (*315*) to pull that record:

In [ ]:
temp = gfs_grb2[315]

Then, using *.values* you can extract the data from the record:

In [ ]:
temp.values

You can also extract the grid information and other import metadata for this record. To see all available information, use the *.keys()* command:

In [ ]:
temp.keys()

The coordinates can be extracted using the *.latitude* and *.longitude*. You can additionally extract the level, units, and forecast time from the file:

In [ ]:
gfs_lat_all = temp.latitudes
gfs_lon_all = temp.longitudes

level = temp.level
units = temp.units

analysis_date = temp.analDate
fcst_time = temp.forecastTime

Problem: The shape of the latitude is MUCH bigger than the temperature... why and what can we do about it?

In [ ]:
temp.values.shape, gfs_lat_all.shape, gfs_lon_all.shape

We can troubleshoot by printing the values. We can see that latitude repeats the values many times.

In [ ]:
gfs_lat_all, gfs_lon_all

A simple way of fixing this is to use np.unique to remove any duplicating values:

In [ ]:
gfs_lat = np.unique(gfs_lat_all)
gfs_lon = np.unique(gfs_lon_all)
gfs_lat.shape, gfs_lon.shape

Now that we know how to import multidimensional data, you will make some plots in the next section.

## Plotting 3-dimensional Data

So far, we have only made line and scatter plots. Matplotlib also supports plotting spatial datasets. However, we often have to do perform several array operations to ensure the x, y, and z coordinates are the same shape. Let's work with a Sea Surface Temperature (SST) dataset in the next example and make a 3D plot.

In [ ]:
import xarray as xr

fname = 'data/sst.mon.ltm.1981-2010.nc'
sst_file_id = xr.open_dataset(fname, engine='netcdf4', decode_times=False)

Like before, you can inspect the contents by typing the variable name:

In [ ]:
sst_file_id

From the printed information above, we can see the following:

* __Dimensions__: The dimensions are named lat, lon, and time, which each have the size of 89, 180, and 12.

* __Coordinates__: Are also lat, lon, and time

* __Variables__: Has three variables, climatology_bounds, sst, valid_yr_count.

Let's import *sst* which is a 2-dimensional variable. You will also need lat and lon, which are both one dimensional:

In [ ]:
sst = sst_file_id["sst"]
sst_lat = sst_file_id["lat"]
sst_lon = sst_file_id["lon"]

Let's inspect the shape and see if the data are already formatted for plotting:

In [ ]:
sst_lat.shape, sst_lon.shape, sst.shape

Contour plots and mesh plots are two useful ways of looking at 3-dimensional data. Both plots require the x, y, and z coordinates to have the same 2D grid. 

The shapes illustrate two problems:
1. SST has a time dependency, we need to just pick one month from the record because we can't plot all 12 on a single graph
2. sst is 2D while lat and lon are 1D. You can use *np.meshgrid()* to project the 1-dimensional x and y coordinates into two dimensions


Problem #1 can be solved by using the xarray select (.sel) command to select just one month, I'll choose December. The index for December is 11 because python numbering starts at 0.

In [ ]:
sst = sst_file_id["sst"].isel(time=11)

The np.meshgrid function will help with problem #2 above. The function is a little confusing at first, so I'll show a simple example. Suppose you have to simple arrays:

In [ ]:
tmp_x = [1,2]
tmp_y = [3,4,5]

*tmp_x* has two elements and *tmp_y* has three. If you create a mesh of the two variables, there will be two variables, both with 3 rows and 2 columns: 

In [ ]:
np.meshgrid(tmp_x, tmp_y)

Returning to the example, below is the meshgrid of the 1-dimensional latitude and longitude coordinates:

In [ ]:
X_sst, Y_sst = np.meshgrid(sst_lon, sst_lat)

Before plotting, you need to check if all the dimensions match. However, after comparing the shape of co to X_co, you can see that the dimensions are flipped:

In [ ]:
sst.shape, X_sst.shape

We've already learned how to use *plt.subplot()* to generate the empty figure (*fig*) and axis (*ax*). 

One line 2, you call *ax.contourf* and input the X_co, Y_co, and transposed co variables. co acts as a color value, which becomes the third dimension of the plot. You then store this object into a variable *co_plot* so that you can pass it into *ax.colorbar* in order to map the colors to numeric values.

In [ ]:
# contourf
fig = plt.figure()
ax = plt.subplot(111)
sst_plot = ax.contourf(X_sst, Y_sst, sst)
fig.colorbar(sst_plot, orientation='horizontal', ax=ax)
plt.show()

Like contour plots, mesh plots are also 2-dimensional plots that display 3-dimensions of information using x, y, coordinates and z for a color scale. However, mesh plots do not perform any smoothing and display data as-is on a regular grid. However, since many satellite datasets are swath-based, irregularly spaced data needs to be re-gridded in order to display it as a mesh grid. In the code block below, let’s compare how the sst data looks using pcolormesh command with the previous example using contour. The code below has no other changes to the plot other than the call to the plot type.

In [ ]:
#pcolormesh
fig = plt.figure()
ax = plt.subplot(111)
sst_plot = ax.pcolormesh(X_sst, Y_sst, sst, shading='auto')
fig.colorbar(sst_plot, orientation='horizontal')
plt.show()

You might notice that there is more structure in the mesh plot than the filled contour. This is useful if you wish to examine fine structure and patterns.

---
**Exercise 2**: Plot 3-dimensional data

Plot *AOD_lat*, *AOD_lon*, and *AOD_500* (which we imported from the "JRR-AOD_v2r3_j01_..." netCDF file as:

1. Check the dimensions for all variables using *.shape*.
2. Do you need to generate a meshgrid with *np.meshgrid()*?
3. Create a contour plot

---
**Solution:**

## Adding Maps to datasets

The package Cartopy add mapping functionality to Matplotlib. Cartopy provides an interface to obtain continent, country, and feature details to overlay onto your plot. Furthermore, Cartopy also enables you to convert your data from one map projection to another, which requires a cartesian coordinate system to the map coordinates. Matplotlib natively supports the six mathematical and map projections (Aitoff, Hammer, Lambert, Mollweide, polar, and rectilinear) and combined with Cartopy, data can be transformed to a total of 33 possible projections.

In [ ]:
from cartopy import crs as ccrs

Just like before, we need to convert the 1D lat and lon coordinates to 2D using meshgrid. We can check the shape to ensure all variables have the same dimensions.

In [ ]:
gfs_temp = temp.values
gfs_x, gfs_y = np.meshgrid(gfs_lon, gfs_lat)

gfs_x.shape, gfs_y.shape, gfs_temp.shape

In [ ]:
fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=ccrs.PlateCarree())

ax.pcolormesh(gfs_x, gfs_y, gfs_temp)

ax.coastlines('50m')
plt.show()

In the next example, you can switch from Plate Carrée to Orthographic. You must define the projection twice, once in the *projection=* keyword and again in the *transform=*. In the *plt.subplot* line, you must define the to coordinates (*ccrs.Orthographic*), which is how you want to axes to show the data.  In the ax.scatter line, you use the transform keyword argument in scatter to define the from coordinates (Plate Carrée), which are the coordinates that the data formatted for.

In [ ]:
fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=ccrs.Orthographic(90, 0))

ax.pcolormesh(gfs_x, gfs_y, gfs_temp, transform=ccrs.PlateCarree())

ax.coastlines('50m')
plt.show()

---
**Exercise 3** Adding maps to plots

Using *sst_lat*, *AOD_lon*, and *AOD_550* (which we imported from the "JRR-AOD_v2r3_j01_..." netCDF file)

1. Create a *pcolormesh* plot
2. Add the coastlines to a standard Plate Caree plot using *projection=* option.

---
**Solution**:

## Summary:

You learned:

* How to import scientific data formats, like netCDF and GRIB2
* Worked with arrays and lists
* How to create a simple maps

Next lesson:
* Obtain datasets from remote sources
* Save data into text and binary files, and plots as images